In [1]:
import pandas as pd
import numpy as np
import naverMap
import datetime
from tqdm import tqdm

#### 데이터 정제
    1. 비발생 농가 시군구 데이터 구하기 (2016년 중 전체 농가 중 발생했던 지역 제외)
    2. 발생 시군구 데이터 구하기 (2016년 발생했던 지역만)
    3. (1. 2.) 에서 뽑은 지역별 철새도래지간 거리 구하기
    
    * 2016년도 기준으로 정제 => 가장 많은 발생건수가 있기 때문!!

In [2]:
occur_df=pd.read_csv('HPAIOccurlist_final.csv', encoding='utf-8-sig')
breeding_df=pd.read_csv("local_breeding.csv", encoding='utf-8-sig')
local_df=pd.read_csv("wild_birds_month.csv", encoding='utf-8-sig')

In [3]:
occur_df.head()

,주소,축종,사육\r\n두수,진단일,발생일,지역코드
0,강원도 원주시 귀래면 주포리,닭-산란계,170000,20210224,20210223,114
1,강원도 인제군 기린면 북리,닭-산란계,28500,20170107,20170103,211
2,강원도 철원군 갈말읍 강포리,닭-산란계,16000,20161212,20161209,95
3,강원도 철원군 갈말읍 강포리,닭-산란계,30000,20161203,20161130,95
4,강원도 춘천시 사북면 지촌리,닭-산란계,48,20080507,20080507,101


In [4]:
# 시군구까지로 지역을 나눔
occur_df['시도']=np.nan
occur_df['시군구']=np.nan
for i in range(len(occur_df)):
    temp=occur_df.loc[i,'주소'].split()
    occur_df.loc[i,'시도']=temp[0] # 시도
    occur_df.loc[i,'시군구']=temp[1] # 시군구

In [5]:
occur_df['발생일']=occur_df['발생일'].astype('str')

In [6]:
occur_df['발생일']=occur_df['발생일'].map(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))

In [7]:
#열 제거
occur_df.drop(['축종','사육\r\n두수','진단일','지역코드'], axis=1, inplace=True)

In [8]:
occur_df['cnt']=1

In [9]:
# 연도별 데이터 만들기
occur_df['year']=occur_df['발생일'].map(lambda x: x.year)

In [10]:
# 15년도 데이터만 가져오기
occur_df_16=occur_df[occur_df['year']==2016].reset_index(drop=True)

In [11]:
occur_df_16.head()

,주소,발생일,시도,시군구,cnt,year
0,강원도 철원군 갈말읍 강포리,2016-12-09,강원도,철원군,1,2016
1,강원도 철원군 갈말읍 강포리,2016-11-30,강원도,철원군,1,2016
2,경기도 광주시 남한산성면 불당리,2016-04-05,경기도,광주시,1,2016
3,경기도 광주시 초월읍 신월리,2016-12-18,경기도,광주시,1,2016
4,경기도 김포시 대곶면 초원지리,2016-12-12,경기도,김포시,1,2016


In [12]:
# 시군구 일부 통합해야함 (세종특별자치시 -> 세종시, 청원군 -> 창원시)
for i in range(len(occur_df_16)):
    if occur_df_16.loc[i,'시도']=='세종특별자치시':
        occur_df_16.loc[i,'시군구']='세종시'
    if occur_df_16.loc[i,'시군구']=='청원군':
        occur_df_16.loc[i,'시군구']='청주시'

In [13]:
# 발생 건수 구하기 (년도 기준)
occur_df_16=occur_df_16.groupby(['시도','시군구','year']).sum()

In [14]:
occur_df_16.reset_index(inplace=True)

In [15]:
# 발생 농가별 건수 complete
occur_df_16.head()

,시도,시군구,year,cnt
0,강원도,철원군,2016,2
1,경기도,광주시,2016,2
2,경기도,김포시,2016,2
3,경기도,안성시,2016,20
4,경기도,양주시,2016,3


In [16]:
occur_df_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      36 non-null     object
 1   시군구     36 non-null     object
 2   year    36 non-null     int64 
 3   cnt     36 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.2+ KB


In [17]:
# 비발생 농가 구하기
breeding_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   년도           533 non-null    float64
 1   주소           532 non-null    object 
 2   시도           533 non-null    object 
 3   시군           532 non-null    object 
 4   축종명          532 non-null    object 
 5   경영체수 합계(건)   533 non-null    float64
 6   시설여부(경영체 수)  534 non-null    object 
 7   Unnamed: 7   534 non-null    object 
 8   경영형태(시설 개수)  534 non-null    object 
 9   Unnamed: 9   534 non-null    object 
 10  사육 규모        533 non-null    float64
 11  단위           533 non-null    object 
dtypes: float64(3), object(9)
memory usage: 50.2+ KB


In [18]:
breeding_df.head()

,년도,주소,시도,시군,축종명,경영체수 합계(건),시설여부(경영체 수),Unnamed: 7,경영형태(시설 개수),Unnamed: 9,사육 규모,단위
0,NaN,NaN,NaN,NaN,NaN,NaN,시설 없음,시설 있음,자영,임차,NaN,NaN
1,2016.0,서울특별시마포구,서울특별시,마포구,육계,1.0,1,0,1,0,1.0,마리
2,2016.0,서울특별시서초구,서울특별시,서초구,육계,2.0,0,2,2,0,65.0,마리
3,2016.0,서울특별시송파구,서울특별시,송파구,육계,1.0,1,0,1,0,2.0,마리
4,2016.0,서울특별시강동구,서울특별시,강동구,육계,1.0,0,1,1,0,15.0,마리


In [19]:
# nan 값 제거
breeding_df=breeding_df.dropna()

In [20]:
#열 제거
breeding_df.drop(['축종명','주소','시설여부(경영체 수)','Unnamed: 7','경영형태(시설 개수)','Unnamed: 9','단위'], 
                 axis=1, inplace=True)

In [21]:
breeding_df.head()

,년도,시도,시군,경영체수 합계(건),사육 규모
1,2016.0,서울특별시,마포구,1.0,1.0
2,2016.0,서울특별시,서초구,2.0,65.0
3,2016.0,서울특별시,송파구,1.0,2.0
4,2016.0,서울특별시,강동구,1.0,15.0
5,2016.0,부산광역시,북구,1.0,60.0


In [22]:
# 전체 건수 구하기
breeding_df_16=breeding_df.groupby(['시도','시군','년도']).sum()

In [23]:
breeding_df_16.reset_index(inplace=True)

In [24]:
breeding_df_16.head()

,시도,시군,년도,경영체수 합계(건),사육 규모
0,강원도,강릉시,2016.0,85.0,132895.0
1,강원도,고성군,2016.0,26.0,97294.0
2,강원도,동해시,2016.0,13.0,42563.0
3,강원도,삼척시,2016.0,32.0,42201.0
4,강원도,속초시,2016.0,8.0,8140.0


In [25]:
#네임명 바꾸기
breeding_df_16=breeding_df_16.rename(columns={'시군':'시군구', '년도':'year', '경영체수 합계(건)':'cnt'})

In [26]:
breeding_df_16['year']=breeding_df_16['year'].astype('int')
breeding_df_16['cnt']=breeding_df_16['cnt'].astype('int')

In [27]:
breeding_df_16.head()

,시도,시군구,year,cnt,사육 규모
0,강원도,강릉시,2016,85,132895.0
1,강원도,고성군,2016,26,97294.0
2,강원도,동해시,2016,13,42563.0
3,강원도,삼척시,2016,32,42201.0
4,강원도,속초시,2016,8,8140.0


In [28]:
breeding_df_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도      194 non-null    object 
 1   시군구     194 non-null    object 
 2   year    194 non-null    int32  
 3   cnt     194 non-null    int32  
 4   사육 규모   194 non-null    float64
dtypes: float64(1), int32(2), object(2)
memory usage: 6.2+ KB


In [29]:
for i in range(len(breeding_df_16)):
    sido=breeding_df_16.loc[i,'시도']
    sigun=breeding_df_16.loc[i,'시군구']
    temp=occur_df_16.query('시도==@sido and 시군구==@sigun').reset_index(drop=True)
    if len(temp)>0: # occur에 존재한다면..
        breeding_df_16.drop([i], axis=0, inplace=True)
breeding_df_16.reset_index(inplace=True, drop=True)

In [30]:
# 비발생 농가별 건수 complete
breeding_df_16.head()

,시도,시군구,year,cnt,사육 규모
0,강원도,강릉시,2016,85,132895.0
1,강원도,고성군,2016,26,97294.0
2,강원도,동해시,2016,13,42563.0
3,강원도,삼척시,2016,32,42201.0
4,강원도,속초시,2016,8,8140.0


In [31]:
breeding_df_16[breeding_df_16['시군구']=='속초시'] # 예시

,시도,시군구,year,cnt,사육 규모
4,강원도,속초시,2016,8,8140.0


In [32]:
#철새도래지 정제
local_df.head()

,구분,시도,시군,14-Oct,14-Nov,14-Dec,15-Jan,15-Feb,15-Mar,15-Oct,...,19-Dec,20-Jan,20-Feb,20-Mar,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar
0,조사종수,-,-,182,190,186,180,186,186,201,...,214,210,167,162,180,180,203,202,178,180
1,합계,-,-,"617,312","1,038,700","1,111,763","974,685","675,607","839,772","483,382",...,"1,817,155","1,629,083","923,296","380,384","575,277","944,700","1,571,494","1,481,717","858,236","472,276"
2,철원평야,강원도,철원군,"51,660","43,900","9,336","10,857","14,109","40,987","70,824",...,"70,858","62,302","99,543","60,365","39,088","36,482","53,663","21,929","54,268","40,268"
3,연천군,경기도,연천군,595,"1,150","1,227","3,530","4,674","3,297","1,349",...,"5,134","7,786",0,0,0,"2,493","3,639","4,452","11,186","2,471"
4,성산-성수,서울특별시,성동구,477,"4,443","7,212","6,333","5,853","21,801","1,434",...,"2,088","2,925",0,0,0,0,"2,312","1,979",0,0


In [33]:
local_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 45 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구분      249 non-null    object
 1   시도      249 non-null    object
 2   시군      249 non-null    object
 3   14-Oct  249 non-null    object
 4   14-Nov  249 non-null    object
 5   14-Dec  249 non-null    object
 6   15-Jan  249 non-null    object
 7   15-Feb  249 non-null    object
 8   15-Mar  249 non-null    object
 9   15-Oct  249 non-null    object
 10  15-Nov  249 non-null    object
 11  15-Dec  249 non-null    object
 12  16-Jan  249 non-null    object
 13  16-Feb  249 non-null    object
 14  16-Mar  249 non-null    object
 15  16-Oct  249 non-null    object
 16  16-Nov  249 non-null    object
 17  16-Dec  249 non-null    object
 18  17-Jan  249 non-null    object
 19  17-Feb  249 non-null    object
 20  17-Mar  249 non-null    object
 21  17-Oct  249 non-null    object
 22  17-Nov  249 non-null    ob

In [34]:
# 의미없는 행 지우기
local_df.drop([0,1], axis=0, inplace=True)

In [35]:
# 의미없는 열 지우기
local_df.drop(['구분'], axis=1, inplace=True)

In [36]:
local_df.reset_index(inplace=True, drop=True)
local_df.head()

,시도,시군,14-Oct,14-Nov,14-Dec,15-Jan,15-Feb,15-Mar,15-Oct,15-Nov,...,19-Dec,20-Jan,20-Feb,20-Mar,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar
0,강원도,철원군,"51,660","43,900","9,336","10,857","14,109","40,987","70,824","52,965",...,"70,858","62,302","99,543","60,365","39,088","36,482","53,663","21,929","54,268","40,268"
1,경기도,연천군,595,"1,150","1,227","3,530","4,674","3,297","1,349","1,986",...,"5,134","7,786",0,0,0,"2,493","3,639","4,452","11,186","2,471"
2,서울특별시,성동구,477,"4,443","7,212","6,333","5,853","21,801","1,434","4,026",...,"2,088","2,925",0,0,0,0,"2,312","1,979",0,0
3,서울특별시,마포구,477,"4,443","7,212","6,333","5,853","21,801","1,434","4,026",...,"2,088","2,925",0,0,0,0,"2,312","1,979",0,0
4,서울특별시,성동구,552,"1,917","2,199","3,023","6,068","5,469",616,"2,945",...,"2,890","2,368",0,0,0,0,"1,918","2,602",0,0


In [37]:
col=list(map(lambda x:'20'+x, local_df.columns[2:].values))

In [38]:
print(col)

['2014-Oct', '2014-Nov', '2014-Dec', '2015-Jan', '2015-Feb', '2015-Mar', '2015-Oct', '2015-Nov', '2015-Dec', '2016-Jan', '2016-Feb', '2016-Mar', '2016-Oct', '2016-Nov', '2016-Dec', '2017-Jan', '2017-Feb', '2017-Mar', '2017-Oct', '2017-Nov', '2017-Dec', '2018-Jan', '2018-Feb', '2018-Mar', '2018-Oct', '2018-Nov', '2018-Dec', '2019-Jan', '2019-Feb', '2019-Mar', '2019-Oct', '2019-Nov', '2019-Dec', '2020-Jan', '2020-Feb', '2020-Mar', '2020-Oct', '2020-Nov', '2020-Dec', '2021-Jan', '2021-Feb', '2021-Mar']


In [39]:
mon={'Oct':'10', 'Nov':'11', 'Dec':'12', 'Jan':'01', 'Feb':'02', 'Mar':'03'}
for i in range(len(col)):
    y=mon[col[i][5:]]
    col[i]=col[i][:5]+y

In [40]:
print(col)

['2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03']


In [41]:
col=list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m'), col))

In [42]:
years=list(map(lambda x: x.year, col))
months=list(map(lambda x: x.month, col))

In [43]:
# 행열 분리 [시도, 시군구, 년도, 월, 철새수 열로 대체]
new_local_df=pd.DataFrame(columns=['시도','시군구','year','month','철새수'])
for i in range(len(local_df)):
    dic={}
    dic['year']=years
    dic['month']=months
    dic['철새수']=local_df.iloc[i,2:].values
    for num in range(len(dic['철새수'])):
        if '시도' not in dic.keys():
            dic['시도']=[]
        if '시군구' not in dic.keys():
            dic['시군구']=[]
        dic['시도'].append(local_df.loc[i,'시도'])
        dic['시군구'].append(local_df.loc[i,'시군'])
    temp=pd.DataFrame.from_dict(dic)
    new_local_df=pd.concat([new_local_df, temp], axis=0)
    new_local_df.reset_index(inplace=True, drop=True)

In [44]:
new_local_df.head()

,시도,시군구,year,month,철새수
0,강원도,철원군,2014,10,"51,660"
1,강원도,철원군,2014,11,"43,900"
2,강원도,철원군,2014,12,"9,336"
3,강원도,철원군,2015,1,"10,857"
4,강원도,철원군,2015,2,"14,109"


In [45]:
new_local_df['year']=new_local_df['year'].astype('int')
new_local_df['month']=new_local_df['month'].astype('int')
new_local_df['철새수']=new_local_df['철새수'].map(lambda x : x.replace(',',''))
new_local_df['철새수']=new_local_df['철새수'].astype('int')

In [46]:
new_local_df.to_csv("month_local.csv", encoding='utf-8-sig', index=False)

In [47]:
# 연도로 합치기
new_local_y=new_local_df.groupby(['시도','시군구','year']).sum()
new_local_y.reset_index(inplace=True)
new_local_y.drop(['month'], axis=1, inplace=True)

In [48]:
# 철새도래지 데이터 정제 완료
new_local_y.head()

,시도,시군구,year,철새수
0,강원도,강릉시,2014,14094
1,강원도,강릉시,2015,43024
2,강원도,강릉시,2016,60737
3,강원도,강릉시,2017,44001
4,강원도,강릉시,2018,52674


In [49]:
# 발생, 비발생 데이터와 철새 도래지간의 거리 구하기
new_local_16=new_local_y[new_local_y['year']==2016].reset_index(drop=True)

In [50]:
new_local_16.head()

,시도,시군구,year,철새수
0,강원도,강릉시,2016,60737
1,강원도,고성군,2016,32419
2,강원도,동해시,2016,43060
3,강원도,삼척시,2016,41257
4,강원도,속초시,2016,29919


In [51]:
new_local_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      126 non-null    object
 1   시군구     126 non-null    object
 2   year    126 non-null    int64 
 3   철새수     126 non-null    int32 
dtypes: int32(1), int64(1), object(2)
memory usage: 3.6+ KB


In [52]:
for i in range(len(new_local_16)):
    if new_local_16.loc[i,'시도']=='세종특별자치시':
        print('check: ', new_local_16.loc[i,'시군구'])
        new_local_16.loc[i,'시군구']='세종시'
    if new_local_16.loc[i,'시군구']=='청원군':
        print('check: ', new_local_16.loc[i,'시군구'])
        new_local_16.loc[i,'시군구']='청주시'

check:  장군면


In [53]:
# 발생농가에서 철새도래지가 속한 시군 까지의 거리 측정
temp=new_local_16['시도']+' '+new_local_16['시군구']
new_local_16['주소']=temp

In [54]:
# 발생/비발생 지역에서 도래지까지의 거리 구하기
for col in new_local_16['주소'].values:
    occur_df_16[col]=np.nan
    breeding_df_16[col]=np.nan

In [55]:
breeding_df_16.head()

,시도,시군구,year,cnt,사육 규모,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
0,강원도,강릉시,2016,85,132895.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강원도,고성군,2016,26,97294.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,강원도,동해시,2016,13,42563.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강원도,삼척시,2016,32,42201.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,강원도,속초시,2016,8,8140.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# 비발생 도래지에서 거리 구하기
for i in tqdm(range(len(breeding_df_16))):
    try:
        start=naverMap.Geocode(breeding_df_16.loc[i,'시도']+' '+breeding_df_16.loc[i,'시군구'])
        for j in range(5, len(breeding_df_16.columns)):
            if breeding_df_16.columns[j]=='세종특별자치시 세종시':
                goal=naverMap.Geocode('세종특별자치시')
            else:
                goal=naverMap.Geocode(breeding_df_16.columns[j])
            breeding_df_16.iloc[i,j]=naverMap.direction(start, goal)
    except:
        print(f'error: ({i},{breeding_df_16.columns[j]})')

100%|████████████████████████████████████████████████████████████████████████████████| 158/158 [32:32<00:00, 12.36s/it]


In [57]:
occur_df_16.head()

,시도,시군구,year,cnt,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
0,강원도,철원군,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,경기도,광주시,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,경기도,김포시,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,경기도,안성시,2016,20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,경기도,양주시,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# 발생 도래지에서 거리 구하기
for i in tqdm(range(len(occur_df_16))):
    try:
        start=naverMap.Geocode(occur_df_16.loc[i,'시도']+' '+occur_df_16.loc[i,'시군구'])
        for j in range(4, len(occur_df_16.columns)):
            if occur_df_16.columns[j]=='세종특별자치시 세종시':
                goal=naverMap.Geocode('세종특별자치시')
            else:
                goal=naverMap.Geocode(occur_df_16.columns[j])
            occur_df_16.iloc[i,j]=naverMap.direction(start, goal)
    except:
        print(f'error: ({i},{occur_df_16.columns[j]})')

 44%|████████████████████████████████████▍                                             | 16/36 [03:20<03:01,  9.06s/it]

error: (15,충청북도 충주시)


100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [07:28<00:00, 12.45s/it]


In [59]:
breeding_df_16.head()

,시도,시군구,year,cnt,사육 규모,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
0,강원도,강릉시,2016,85,132895.0,0.00,78.48,32.87,42.18,56.38,...,197.29,215.72,187.49,254.14,141.90,138.31,157.08,161.77,174.11,119.19
1,강원도,고성군,2016,26,97294.0,78.48,0.00,110.76,120.21,22.11,...,218.98,237.07,211.02,263.87,184.10,174.16,193.82,192.42,211.68,161.70
2,강원도,동해시,2016,13,42563.0,32.87,110.76,0.00,9.45,88.66,...,204.20,222.06,194.01,264.25,141.63,141.82,158.78,166.28,174.38,120.75
3,강원도,삼척시,2016,32,42201.0,42.18,120.21,9.45,0.00,98.11,...,205.31,222.93,195.06,266.07,141.10,142.38,158.67,166.93,173.75,121.00
4,강원도,속초시,2016,8,8140.0,56.38,22.11,88.66,98.11,0.00,...,210.92,229.32,202.36,259.58,170.27,161.72,181.49,181.62,199.38,147.39


In [60]:
breeding_df_16.to_csv("non_occur_distance.csv", encoding='utf-8-sig', index=False)

In [61]:
occur_df_16.head()

,시도,시군구,year,cnt,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
0,강원도,철원군,2016,2,143.85,104.08,172.61,180.20,111.94,114.51,...,153.35,168.18,149.12,179.76,153.83,138.22,153.23,144.01,167.99,139.39
1,경기도,광주시,2016,2,147.25,150.03,164.40,168.65,145.83,139.80,...,74.54,90.85,69.44,113.97,82.97,66.68,77.25,65.80,89.93,76.85
2,경기도,김포시,2016,2,190.71,175.55,211.64,216.78,177.27,174.80,...,95.23,104.54,95.72,103.55,130.04,114.35,120.00,105.85,128.08,127.60
3,경기도,안성시,2016,20,163.50,184.99,172.20,174.00,176.45,167.43,...,34.58,53.12,26.03,92.08,49.90,37.26,36.51,21.91,44.71,57.42
4,경기도,양주시,2016,3,160.89,140.96,184.38,190.35,143.36,141.71,...,110.74,124.09,108.05,133.22,126.20,109.89,120.87,108.99,133.00,117.67


In [69]:
occur_df_16.query("시군구=='세종시'")

,시도,시군구,year,cnt,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
15,세종특별자치시,세종시,2016,17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
start=naverMap.Geocode(occur_df_16.loc[15,'시도'])
for j in range(4, len(occur_df_16.columns)):
    if occur_df_16.columns[j]=='세종특별자치시 세종시':
        goal=naverMap.Geocode('세종특별자치시')
    else:
        goal=naverMap.Geocode(occur_df_16.columns[j])
    occur_df_16.iloc[15,j]=naverMap.direction(start, goal)

In [71]:
occur_df_16.query("시군구=='세종시'")

,시도,시군구,year,cnt,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
15,세종특별자치시,세종시,2016,17,199.46,235.53,199.36,198.48,223.88,212.84,...,42.91,45.54,40.39,93.28,57.93,62.4,42.77,43.7,25.35,80.27


In [72]:
occur_df_16.head()

,시도,시군구,year,cnt,강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군,...,충청남도 아산시,충청남도 예산군,충청남도 천안시,충청남도 태안군,충청북도 괴산군,충청북도 음성군,충청북도 증평군,충청북도 진천군,충청북도 청주시,충청북도 충주시
0,강원도,철원군,2016,2,143.85,104.08,172.61,180.20,111.94,114.51,...,153.35,168.18,149.12,179.76,153.83,138.22,153.23,144.01,167.99,139.39
1,경기도,광주시,2016,2,147.25,150.03,164.40,168.65,145.83,139.80,...,74.54,90.85,69.44,113.97,82.97,66.68,77.25,65.80,89.93,76.85
2,경기도,김포시,2016,2,190.71,175.55,211.64,216.78,177.27,174.80,...,95.23,104.54,95.72,103.55,130.04,114.35,120.00,105.85,128.08,127.60
3,경기도,안성시,2016,20,163.50,184.99,172.20,174.00,176.45,167.43,...,34.58,53.12,26.03,92.08,49.90,37.26,36.51,21.91,44.71,57.42
4,경기도,양주시,2016,3,160.89,140.96,184.38,190.35,143.36,141.71,...,110.74,124.09,108.05,133.22,126.20,109.89,120.87,108.99,133.00,117.67


In [73]:
occur_df_16.to_csv("occur_distance.csv", encoding='utf-8-sig', index=False)

#### 철새도래지에서 멀어질수록 발생지역 보다 비발생지역의 발생건수가 더 많다.
    1.  철새 도래지에서 비발생지역의 최소거리보다 가까운 발생지역이 더 많은지 검토 하면 됨. (0보다 카운터 숫자가 많으면 됨)

In [90]:
# 예시
occ_col1=occur_df_16[['시도','시군구','충청남도 아산시']].sort_values(['충청남도 아산시']).reset_index(drop=True)

In [91]:
occ_col1.head()

,시도,시군구,충청남도 아산시
0,충청남도,아산시,0.00
1,충청남도,천안시,10.31
2,경기도,평택시,24.54
3,경기도,안성시,34.58
4,충청북도,진천군,39.23


In [92]:
non_col1=breeding_df_16[['시도','시군구','충청남도 아산시']].sort_values(['충청남도 아산시']).reset_index(drop=True)

In [93]:
non_col1.head()

,시도,시군구,충청남도 아산시
0,충청남도,예산군,18.57
1,충청남도,당진시,33.62
2,충청남도,홍성군,36.99
3,충청남도,공주시,39.58
4,경기도,오산시,40.57


In [98]:
# 철새 도래지에서 비발생지역의 최소거리보다 가까운 발생지역이 더 많은지 검토 하면 됨.
cnt_occur_df=pd.DataFrame(index=breeding_df_16.columns[5:],columns=['cnt'])
for col in breeding_df_16.columns[5:]:
    occ_col=occur_df_16[['시도','시군구',col]].sort_values([col]).reset_index(drop=True)
    non_col=breeding_df_16[['시도','시군구',col]].sort_values([col]).reset_index(drop=True)
    min_st=non_col.loc[0,col] # 최소거리
    cnt=0
    for i in range(len(occ_col)):
        if occ_col.loc[i,col]<=min_st:
            cnt+=1
        else:
            break
    cnt_occur_df.loc[col,'cnt']=cnt

In [103]:
# 철새도래지에서 비발생지역이 발생지역보다 더 가까웠던 지역 수
len(cnt_occur_df.query("cnt==0"))

91

In [105]:
# 철새도래지에서 발생지역이 비발생지역보다 더 가까웠던 지역 수
len(cnt_occur_df.query("cnt>0"))

35

In [106]:
cnt_occur_df.query("cnt>0")

,cnt
강원도 철원군,1
경기도 광주시,1
경기도 김포시,2
경기도 안성시,3
경기도 양평군,2
경기도 여주시,3
경기도 용인시,1
경기도 이천시,5
경기도 처인구,1
경기도 평택시,2


In [107]:
da=pd.read_excel('HPAI_발생모음(2000~2021).xlsx')

In [109]:
da.drop(['축주명','번호','진단명','진단일','기관','신규/\n추가','시스템\n구분'], axis=1, inplace=True)

In [111]:
da['감염\n두수']=da['감염\n두수']+da['폐사\n두수']

In [114]:
da.drop(['폐사\n두수'], axis=1, inplace=True)

In [116]:
da.to_csv("초기데이터.csv", encoding='utf-8-sig', index=False)

In [118]:
da.drop(['사육\n두수','검사\n두수'], axis=1, inplace=True)

In [122]:
da=da.dropna()

In [124]:
da.reset_index(inplace=True, drop=True)

In [126]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주소      496 non-null    object 
 1   축종      496 non-null    object 
 2   감염
두수   496 non-null    float64
 3   발생일     496 non-null    object 
dtypes: float64(1), object(3)
memory usage: 15.6+ KB


In [127]:
for i in range(len(da)):
    if da.loc[i,'감염\n두수']==0:
        da.drop([i], axis=0, inplace=True)
da.reset_index(inplace=True, drop=True)

C:\Users\user\anaconda3\envs\minipj\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [133]:
da=da.rename(columns={'감염\n두수':'감염두수'})

In [135]:
re=pd.read_csv("HPAIOccurlist_final.csv", encoding='utf-8-sig')

In [136]:
re

,주소,축종,사육\r\n두수,진단일,발생일,지역코드
0,강원도 원주시 귀래면 주포리,닭-산란계,170000,20210224,20210223,114
1,강원도 인제군 기린면 북리,닭-산란계,28500,20170107,20170103,211
2,강원도 철원군 갈말읍 강포리,닭-산란계,16000,20161212,20161209,95
3,강원도 철원군 갈말읍 강포리,닭-산란계,30000,20161203,20161130,95
4,강원도 춘천시 사북면 지촌리,닭-산란계,48,20080507,20080507,101
...,...,...,...,...,...,...
1117,충청북도 청주시 흥덕구 오송읍 오송리,닭-육계,1500,20161206,20161204,131
1118,충청북도 충주시 대소원면 만정리,닭-토종닭,100,20161208,20161206,127
1119,충청북도 충주시 동량면 대전리,오리-종오리,7600,20210203,20210202,127
1120,충청북도 충주시 소태면 주치리,닭-산란계,550000,20210313,20210311,127


In [137]:
da['발생일']=da['발생일'].map(lambda x: x.replace('-',''))

In [140]:
da

,주소,축종,감염두수,발생일,지역코드
0,전라북도 정읍시 소성면 기린리,오리-육용오리,3.0,20201126,NaN
1,충청남도 아산시 둔포면 염작리,닭-산란계,10.0,20180317,NaN
2,충청남도 천안시 서북구 성환읍 양령리,닭-산란계,164.0,20180208,NaN
3,경기도 화성시 팔탄면 구장리,닭-산란계,800.0,20180126,NaN
4,경기도 포천시 영북면 자일리,닭-산란계,64890.0,20180103,NaN
...,...,...,...,...,...
458,경상북도 경주시 안강읍 육통리,닭-산란계,11450.0,20031221,NaN
459,경상북도 경주시 안강읍 육통리,닭-산란계,27000.0,20031224,NaN
460,경상북도 경주시 안강읍 육통리,닭-산란계,27000.0,20031224,NaN
461,경기도 이천시 율면 신추리,닭-산란계,200.0,20031225,NaN


In [139]:
da['지역코드']=np.nan

In [150]:
for i in range(len(da)):
    try:
        lo=da.loc[i,'주소']
        df=re.query('주소==@lo').reset_index(drop=True)
        da.loc[i,'지역코드']=int(df.loc[0,'지역코드'])
    except:
        print('error: ', i)

error:  223
error:  230
error:  267
error:  302


In [154]:
da

주소      세종특별자치시  소정면 소정리
축종                 닭-산란계
감염두수            261600.0
발생일             20140325
지역코드                 NaN
Name: 223, dtype: object